## <font color='darkblue'>503. Next Greater Element II</font>
Given a circular integer array `nums` (i.e., the next element of `nums[nums.length - 1]` is `nums[0]`), return the <b>next greater number for every element in nums</b>.

The next greater number of a number x is the first greater number to its traversing-order next in the array, which means you could search circularly to find its next greater number. If it doesn't exist, return -1 for this number.

* **Example 1:**
> Input: nums = \[1,2,1\] <br/>
> Output: \[2,-1,2\] <br/>
> Explanation: The first 1's next greater number is 2; <br/>
> The number 2 can't find next greater number.  <br/>
> The second 1's next greater number needs to search circularly, which is also 2. <br/>

* **Example 2:**
> Input: nums = \[1,2,3,4,3\] <br/>
> Output: \[2,3,4,-1,4\] <br/>

In [1]:
from typing import List

In [2]:
def evaluate_solution(sol):
    for data, ans in [
        ([1,2,1], [2,-1,2]),
        ([1,2,3,4,3], [2,3,4,-1,4]),
    ]:
        rel = sol.nextGreaterElements(data)
        assert ans == rel
        
    print("Pass!")

In [3]:
import random
large_data = random.sample(range(-500000, 500000), 500000)

## <font color='darkblue'>Solution</font>
底下我們要來看幾個解法, 首先是 Brute force 的方法, 接著是透過 Stack 的優化解.

<a id='sol_1'></a>
### <font color='darkgreen'>Brute force approach</font>
Brute force 的解法的時間複雜度是 $O(n^2)$ , 好處是解法易懂. 這邊我們使用 generator (註一) 來幫我們 iterate 從 `nums` 的某個位置遍循整個串列:

In [4]:
def iterate_nums(nums, pos):
    num_size = len(nums)
    for _ in range(num_size):
        pos = pos % num_size
        yield nums[pos]
        pos += 1

In [5]:
nums = [i+1 for i in range(5)]
print(f'nums = {nums}')
# Iterate from pos 1:
for i in range(len(nums)):
    print(f"Iterate from pos={i}: {list(iterate_nums(nums, i))}")

nums = [1, 2, 3, 4, 5]
Iterate from pos=0: [1, 2, 3, 4, 5]
Iterate from pos=1: [2, 3, 4, 5, 1]
Iterate from pos=2: [3, 4, 5, 1, 2]
Iterate from pos=3: [4, 5, 1, 2, 3]
Iterate from pos=4: [5, 1, 2, 3, 4]


接著解法的描述如下:
* 遍循 `nums` - `for i, v in nums:`
  * 從 `i+1` 的位置上往前找第一個比 `v` 大的數值
  * 如果存在就把該數字存回 `ans` , 如果不存在就存 -1 進 `ans`

In [6]:
class Solution:
    def nextGreaterElements(self, nums: List[int]) -> List[int]:                            
        ans = []
        
        for i, v in enumerate(nums):
            is_found = False
            for next_v in iterate_nums(nums, i+1):
                if next_v > v:
                    ans.append(next_v)
                    is_found = True
                    break
                    
            if not is_found:
                ans.append(-1)
        
        return ans

In [7]:
sol = Solution()
evaluate_solution(sol)

Pass!


看起來解法是對的, 接著來看看處理大資料的時間 (約 2s):

In [8]:
%%time
ans = sol.nextGreaterElements(large_data)

CPU times: user 1.88 s, sys: 3.87 ms, total: 1.89 s
Wall time: 1.89 s


### <font color='darkgreen'>Optimized solution by applying Stack</font>
網路可以找到 <a href='https://blog.csdn.net/fuxuemingzhu/article/details/79463006'>這篇</a> 解法, Leetcode 裡也有給出解法 (<a href='https://leetcode.com/problems/next-greater-element-ii/solution/'>link</a>). 我們都知道 <b><a href='https://en.wikipedia.org/wiki/Stack_(abstract_data_type)'>Stack</a></b> (註二) 是一個 LIFO (Last in, first out) 的資料結構:
![stack gif](https://miro.medium.com/max/1280/1*lb-0r80YYhcnoVcQ3HY-1g.gif)
<br/>
([image source](https://medium.com/@todoroski97/data-structure-stack-17b80ed3bfa9))

那我們怎麼使用 Stack 來有效的處理這個問題? 從 <b><a href='#sol_1'>Brute force</a></b> 的解法我們知道最花時間是往前去找比目前數字大的第一個數字, 因此一個想法是是否我們能將上一次的搜尋結果存到 Stack, 如此在下一次的搜尋我們直接去 Stack 找 (只保留目前找到比當前大的數字) 而不用遍循整個 `nums`. 說起來有點抽象, 下面直接來看範例, 考慮我們有一個 `nums = [3, 8, 4, 1, 2]`:

In [9]:
stack = []  # 準備一個 stack 來存放上一次的搜尋找到的大數字
nums = [3, 8, 4, 1, 2] 
ans = [-1] * len(nums)  # 用來搜尋結果, 因為還沒開始找, 所以預設都是 -1

def print_status():
    print(f'stack={stack}; ans={ans}')

In [10]:
# Round 1: 目前 pos=4, num=2, stack=[], 因為 stack 是空的, 所以沒有 ans, 然後把 2 放進 stack
stack = [2]
print_status()

stack=[2]; ans=[-1, -1, -1, -1, -1]


In [11]:
# Round 2: 目前 pos=3, num=1, stack = [2], stack 最上面的數字比目前數字 1 大, 
#          因此把 2 放到 ans[3]=2, 然後把 1 放進 stack = [2, 1]
stack = [2, 1]
ans[3] = 2
print_status()

stack=[2, 1]; ans=[-1, -1, -1, 2, -1]


In [12]:
# Round 3: 目前 pos=2, num=4, stack = [2, 1], 接著我們開始從 stack 找比 4 還要大的數字, 
#          如果遇到小於等於 4 的數字, 就把它從 stack 移除, 最後再把 4 放進去 stack
num, pos = 4, 2
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 4 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 4 放進去 stack
stack.append(num)
print_status()

stack=[4]; ans=[-1, -1, -1, 2, -1]


In [13]:
# Round 4: 目前 pos=1, num=8, stack = [4], 接著我們開始從 stack 找比 8 還要大的數字, 
#          如果遇到小於等於 8 的數字, 就把它從 stack 移除, 最後再把 8 放進去 stack
num, pos = 8, 1
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 8 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 8 放進去 stack
stack.append(num)
print_status()

stack=[8]; ans=[-1, -1, -1, 2, -1]


In [14]:
# Round 5: 目前 pos=0, num=3, stack = [8], 接著我們開始從 stack 找比 3 還要大的數字, 
#          如果遇到小於等於 3 的數字, 就把它從 stack 移除, 最後再把 3 放進去 stack
pos = 0
num = nums[pos]
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 3 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 3 放進去 stack
stack.append(num)
print_status()

Replace ans[0] with 8
stack=[8, 3]; ans=[8, -1, -1, 2, -1]


到目前為止, 我們已經從遍循整個 `nums` 一次, 但是我們還要繼續遍循一次, 因為再處理 pos>1 的數字時, 它們在被處理時, stack 並沒有看到這些數字之前的數字 (e.g. 在處理 pos=4, num=2 時, stack 是空的), 處理過一輪後, 目前 stack 已經有看過所有數字並保留目前位置上往前看到的最大數字.

In [15]:
# Round 6: 目前 pos=4, num=2, stack = [8, 3], 接著我們開始從 stack 找比 2 還要大的數字, 
#          如果遇到小於等於 2 的數字, 就把它從 stack 移除, 最後再把 2 放進去 stack
pos = 4
num = nums[pos]
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 3 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 3 放進去 stack
stack.append(num)
print_status()

Replace ans[4] with 3
stack=[8, 3, 2]; ans=[8, -1, -1, 2, 3]


In [16]:
# Round 7: 目前 pos=3, num=1, stack=[8, 3, 2], 因為 ans[3] = 2 > -1, 所以這一輪不用處理

In [17]:
# Round 8: 目前 pos=2, num=4, stack=[8, 3, 2], 接著我們開始從 stack 找比 4 還要大的數字, 
#          如果遇到小於等於 4 的數字, 就把它從 stack 移除, 最後再把 4 放進去 stack
pos = 2
num = nums[pos]
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 3 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 3 放進去 stack
stack.append(num)
print_status()

Replace ans[2] with 8
stack=[8, 4]; ans=[8, -1, 8, 2, 3]


In [18]:
# Round 9: 目前 pos=1, num=8, stack=[8, 3, 2], 接著我們開始從 stack 找比 8 還要大的數字, 
#          如果遇到小於等於 8 的數字, 就把它從 stack 移除, 最後再把 8 放進去 stack
pos = 1
num = nums[pos]
while stack:
    if stack[-1] <= num:
        stack.pop()
    else:
        break
        
if stack:  # stack 不是空的, 說明有找到比 3 大的數字
    print(f'Replace ans[{pos}] with {stack[-1]}')
    ans[pos] = stack[-1]
    
# 把 3 放進去 stack
stack.append(num)
print_status()

stack=[8]; ans=[8, -1, 8, 2, 3]


In [19]:
# Round 10: 目前 pos=0, num=3, stack=[8], 因為 ans[0] = 8 > -1, 所以這一輪不用處理 (Last round)

所以根據上面的解說, 將上述演算法實作的結果如下:

In [20]:
class Solution:
    def nextGreaterElements(self, nums: List[int]) -> List[int]:                            
        nums_length = len(nums)
        ans, stack = [-1] * nums_length, []
        
        for _ in range(2):
            for pos in range(nums_length-1, -1, -1):
                val = nums[pos]
                while stack and stack[-1] <= val:
                    stack.pop()
                    
                if stack:
                    ans[pos] = stack[-1]
                    
                stack.append(val)
        
        return ans

In [21]:
sol = Solution()
evaluate_solution(sol)

Pass!


看起來我們的實作是對的, 接著來看處理大資料所需時間 (550ms~450ms << 2s):

In [22]:
%%time
ans = sol.nextGreaterElements(large_data)

CPU times: user 461 ms, sys: 167 µs, total: 461 ms
Wall time: 461 ms


根據 Leetcode 的執行結果:
> Runtime: 224 ms, faster than 63.42% of Python3 online submissions for Next Greater Element II.  <br/>
> Memory Usage: 15.8 MB, less than 40.35% of Python3 online submissions for Next Greater Element II.

## <font color='darkblue'>Supplement</font>
* <a href='https://realpython.com/introduction-to-python-generators/'>1. RealPython - How to Use Generators and yield in Python</a>
* <a href='https://realpython.com/how-to-implement-python-stack/'>2. RealPython - How to Implement a Python Stack</a>